In [135]:
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import numpy as np
#import pandas as pd

In [173]:
driver = webdriver.Chrome()

In [ ]:
driver.get('https://www.internationalstudent.com/schools_awarding_aid/NY/New%20York.html')
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [98]:
table = soup.find('table')
lines = table.find_all('tr')
line = lines[1]
print(line)

<tr data-key="174"><td><a href="http://www.adelphi.edu">Adelphi University <i class="fa fa-external-link"></i></a></td><td>$32,500.00</td><td>333</td><td></td><td>$14,400.00</td></tr>


In [131]:
# process different format of # Awarded
def get_intl_awarded(text, total_intl):
    if text == 'All':
        text = '100%'
    if re.search('%',text):
        pct = int(re.sub('%','',text))
        intl_awarded = int(total_intl * pct /100)
    elif text == '':
        intl_awarded = 0
    else:
        intl_awarded = int(text)
    return intl_awarded

text = lines[6].find_all('td')[3].text
total_intl = 500
get_intl_awarded(text, total_intl)

53

In [183]:
# record entries in one line of the table
def get_entry(line):
    entry = {}
    entry['state_init'] = state_init
    entry['state_name'] = state_name
    entry['inst_name'] = line.td.text.strip()
    entry['link'] = line.td.a['href']
    entries = line.find_all('td')
    annual_cost_raw = entries[1].text[1:-3]
    entry['annual_cost'] = int(re.sub(',','',annual_cost_raw))
    total_intl = int(re.sub(',','',entries[2].text))
    entry['total_intl'] = total_intl
    try:
        entry['intl_awarded'] = get_intl_awarded(entries[3].text, total_intl)
    except:
        entry['intl_awarded'] = np.NaN
        print(line, 'something went wrong')
    avg_award_raw = line.find_all('td')[4].text[1:-3]
    entry['avg_award'] = int(re.sub(',','',avg_award_raw))
    return(entry)



IndexError: list index out of range

In [100]:
state_init = 'NY'
state_name = 'New York'
state_name_link = re.sub(' ', '%20', state_name)
state_name_link
url = ('https://www.internationalstudent.com/schools_awarding_aid/'+state_init+'/'+state_name_link+'.html')
url

'https://www.internationalstudent.com/schools_awarding_aid/NY/New%20York.html'

In [170]:
# record the whole table of a state
def get_state(state_init, state_name):
    state_name_link = re.sub(' ', '%20', state_name)
    url = ('https://www.internationalstudent.com/schools_awarding_aid/'+state_init+'/'+state_name_link+'.html')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml') 
    table = soup.find('table')
    lines = table.find_all('tr')
    entries = []
    if len(lines) > 2:
        for line in lines[1:]:
            entries.append(get_entry(line))
    elif len(lines) == 2:
        if lines[1].text == 'No results found.':
            return(entries)
        else: # it means there is one school
            entries.append(get_entry(lines[1]))
    else:
        print('something went wrong', state_name)
    return(entries)


In [150]:
with open('Data/states.txt','r') as file:
    states = file.readlines()
    #states.append(line)
print(states[0])

				<option value="AL">Alabama</option>



'Alabama'

In [178]:
def get_state_name(state):
    state_init = state[19:21]
    state_name = re.search('>\D+<',state).group()[1:-1]
    return state_init, state_name
get_state_name(states[0])

('AL', 'Alabama')

In [181]:
state_init, state_name = get_state_name(states[8])
#get_state('De', 'Delaware')
state_name

'District Of Columbia'

True

In [182]:
# use the state initials and names to complete the url
all_institutions = []
for state in states:
    state_init, state_name = get_state_name(state)
    inst_info = get_state(state_init, state_name)
    all_institutions.extend(inst_info)
    time.sleep(np.random.lognormal(0.75) + 0.2)

ValueError: invalid literal for int() with base 10: 'na'

In [185]:
for state in states[9:]:
    state_init, state_name = get_state_name(state)
    inst_info = get_state(state_init, state_name)
    all_institutions.extend(inst_info)
    time.sleep(np.random.lognormal(0.75) + 0.2)

<tr data-key="52"><td><a href="http://www.ucf.edu">University of Central FL <i class="fa fa-external-link"></i></a></td><td>$26,530.00</td><td>902</td><td>na</td><td>$763.00</td></tr> something went wrong
<tr data-key="130"><td><a href="http://www.smfa.edu">School of the Museum of Fine Arts, Boston <i class="fa fa-external-link"></i></a></td><td>$39,860.00</td><td>77</td><td>16 of 20</td><td>$16,000.00</td></tr> something went wrong
<tr data-key="144"><td><a href="http://www.calvin.edu/admissions/intlstud.htm">Calvin College <i class="fa fa-external-link"></i></a></td><td>$30,300.00</td><td>372</td><td> N/A</td><td>$13,700.00</td></tr> something went wrong
<tr data-key="322"><td><a href="http://www.culver.edu">Culver Stockton College <i class="fa fa-external-link"></i></a></td><td>$23,700.00</td><td>5</td><td>N/A</td><td>$15,000.00</td></tr> something went wrong
<tr data-key="335"><td><a href="http://www.ship.edu">Shippensburg University of Pennsylvania <i class="fa fa-external-link"><

In [186]:
import pandas as pd
institution_info = pd.DataFrame(all_institutions)
institution_info.to_csv('Data/InternationalStudent_com.csv', index = False)